In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from NoiseKD import Teacher, ToyTransformer, slm_init_config, slm_model_config, count_parameters

In [57]:
#SLM = SimpleLanguageModel(**slm_init_config) #THIS GRADIENT ACCUMULATES!!!!BUGGGGGG 

In [10]:
embedding_dim = 16
num_heads = 8
hidden_dim  = 11
num_layers = 2
dropout = 0.1
vocab_size = 80
class_num = vocab_size
batch_size = 50
sequence_length = 160

In [13]:
TT = ToyTransformer(vocab_size, embedding_dim, num_heads, hidden_dim, num_layers, dropout,sequence_length)
teacher_toy = Teacher(TT,(sequence_length,))

In [14]:
teacher_toy.load_state_dict('good_toy_5.pth')

Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)
lets try ints!


In [15]:
#we want to have a configured teacher, then be able to duplicate it with the embedding layer converted to something
#that can take floats.
train_args = { 'val_train' : "train"
                      , 'n' : 100_000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size']
                      , 'std': 1.0
        }
teacher_toy.generate_data(**train_args)

val_args = { 'val_train' : "val"
                      , 'n' : 100_000
                      , 'dist_type' : 'ints'
                      , 'm' : slm_init_config['vocab_size']
                      , 'std': 1.0
        }
teacher_toy.generate_data(**val_args)

In [17]:
embedding_weight_tensor = teacher_toy.model.embedding.weight.detach() #.numpy().shape

In [19]:
embedding_layer = teacher_toy.model.embedding

In [20]:
embedding_weight_tensor

tensor([[ 1.2303, -1.1401, -2.3739,  ...,  1.1756,  0.0569, -1.4886],
        [-0.0173,  0.6813,  1.9396,  ..., -0.1121, -1.2799,  0.0506],
        [-1.6924, -0.7148,  0.7489,  ...,  0.1464,  2.0502, -0.7353],
        ...,
        [ 0.3459,  1.0686,  1.5207,  ...,  0.5726, -0.2457,  0.6735],
        [ 0.2913,  0.5065,  0.6570,  ..., -0.0601,  1.4018,  2.1171],
        [-0.5423, -0.4840,  0.7068,  ..., -0.3819, -1.2273, -2.1977]])

In [22]:
test_data = teacher_toy.train_inputs[0:10]#this is a batch of 10 inputs, indexes.  
test_data.shape #this is batch of 10, seq_len of 160, each corresponding to an index.  
#so, if we do onehot, it should be b x 160 x vocab_size which is 80 in this case.  

torch.Size([10, 160])

In [23]:
embedding_layer(test_data).shape#works.  slm_init_config['embedding_dim'] is where the 16 is coming from.

torch.Size([10, 160, 16])

In [24]:
embedding_weight_tensor.shape #with onehot, it should be b x 160 x 80 * 80 x 16 = b x 160 x 16.  just linear! 1d convolution

torch.Size([80, 16])

In [58]:
def batch_one_hot(input_sequences, vocab_size):
    batch_size = input_sequences.size(0)
    max_seq_length = input_sequences.size(1)
    
    # Create a tensor to store the one-hot encodings
    one_hot_input = torch.zeros(batch_size, max_seq_length, vocab_size)
    
    # Use scatter_ to set the appropriate elements to 1 in each batch
    one_hot_input.scatter_(2, input_sequences.unsqueeze(2), 1)
    
    return one_hot_input

oh = batch_one_hot(test_data, vocab_size = 80)
##test_data == torch.argmax(oh, dim=2)

In [59]:
def Linearize_Embedding(embedding_layer):
    embedding_weight_tensor = embedding_layer.weight.detach() 
    shape = embedding_weight_tensor.shape
    vocab_size = shape[0]
    embedding_dim = shape[1]
    lin = nn.Linear(vocab_size,embedding_dim, bias = False)
    #print(lin.weight.shape)
    #print(embedding_weight_tensor.shape)
    lin.weight = nn.Parameter(embedding_weight_tensor.T) #not sure about this transpose
    return lin
L = Linearize_Embedding(embedding_layer)


In [60]:
##also for future reference, it sort of looks like linear layers just operate across the last layer. i may not need to stack.

In [61]:
oh.shape #this is 10 batches, 160 seq_len, and 80 embedding dim.  


torch.Size([10, 160, 80])

In [62]:
embedding_layer(test_data).shape,L(oh).shape
#normal takes the test data (ints), linearized takes onehotted data.

(torch.Size([10, 160, 16]), torch.Size([10, 160, 16]))

In [63]:
#embedding_layer(test_data),L(oh)

In [64]:
torch.sum((L(oh) - embedding_layer(test_data))**2) #ha.  it was the bias.  

tensor(0., grad_fn=<SumBackward0>)

In [65]:
nn.Parameter(embedding_weight_tensor) == L.weight.T #init works. not sure about having a bias in linear layer tho...

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [66]:
import copy
copy_TT = copy.deepcopy(TT)
copy_TT.embedding = L #swap out the embedding
TT,copy_TT

(ToyTransformer(
   (embedding): Embedding(80, 16)
   (transformer_encoder): TransformerEncoder(
     (layers): ModuleList(
       (0): TransformerEncoderLayer(
         (self_attn): MultiheadAttention(
           (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
         )
         (linear1): Linear(in_features=16, out_features=11, bias=True)
         (dropout): Dropout(p=0.1, inplace=False)
         (linear2): Linear(in_features=11, out_features=16, bias=True)
         (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
         (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
         (dropout1): Dropout(p=0.1, inplace=False)
         (dropout2): Dropout(p=0.1, inplace=False)
       )
       (1): TransformerEncoderLayer(
         (self_attn): MultiheadAttention(
           (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
         )
         (linear1): Linear(in_features=16, out_featur

In [71]:
TT(test_data[0:10]) #,test_data[0:10] ##accumulating gradient!!!!!!

tensor([[6.3937e-34, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 2.0102e-40, 0.0000e+00, 1.2086e-23, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 4.9053e-32, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5596e-33,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         4.2396e-12, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.4853e-39,
         0.0000e+00, 6.3547e-31, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e

In [72]:
oh = batch_one_hot(test_data, vocab_size = 80) #i do this above, this is just so i can remeber what it is doing
copy_TT(oh[0:10]) #now it takes one hot, not indexes.

tensor([[6.3937e-34, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 2.0102e-40, 0.0000e+00, 1.2086e-23, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 4.9053e-32, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5596e-33,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         4.2396e-12, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 4.4853e-39,
         0.0000e+00, 6.3547e-31, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e

In [73]:
torch.sum((TT(test_data) - copy_TT(oh))**2) #asdflkajsdglfonsdg;aogh so the parameter layer is working, but the final bit isnt

tensor(0., grad_fn=<SumBackward0>)

In [74]:
# final check, is the batch bug fixed"
TT(test_data[0:10])[3:4],TT(test_data[3:4]) ##just floating point!!!!

(tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.9026e-42, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.7119e-40, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.7021e-38,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          1.9527e-07, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 2.0291e-42, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000

In [76]:
copy_TT(oh[0:10])[3:4],copy_TT(oh[3:4])

(tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.9026e-42, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.7119e-40, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.7021e-38,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          1.9527e-07, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 2.0291e-42, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000